# LangChain Conversation with Memory 🧠💬

This notebook demonstrates how to create a conversational AI using LangChain, incorporating memory to maintain context throughout the conversation. We'll be using `ConversationSummaryMemory` to summarize the conversation history, allowing the AI to remember past turns without needing to store the entire transcript.

We'll also use `SQLChatMessageHistory` to persist the conversation history in a SQLite database, so you can pick up your conversation later! 💾

Let's dive in! 👇

In [ ]:
# Install the required libraries
!pip install langchain langchain-openai langchain-community SQLAlchemy

## Setting up the OpenAI API Key 🔑

To use LangChain with OpenAI's models, you need an OpenAI API key. We'll securely input the key using `getpass` to prevent it from being visible in the notebook output.

**Note:** Make sure you have your OpenAI API key ready!

In [8]:
import os
from getpass import getpass

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


## Initializing the Language Model (LLM) 🤖

We'll use the `OpenAI` class from `langchain_openai` to initialize our language model. The `temperature` parameter controls the randomness of the AI's responses. A temperature of 0 means the AI will be more deterministic.

In [9]:
from langchain_openai import OpenAI

# Initialize the language model
llm = OpenAI(temperature=0)

## Setting up Conversation Memory with SQLite 🧠💾

Here, we set up the conversation memory using `ConversationSummaryMemory` and `SQLChatMessageHistory`.

- `SQLChatMessageHistory` stores the conversation history in a SQLite database file (`langchain_memory.db`). This allows us to persist the conversation across sessions.
- `ConversationSummaryMemory` takes the chat history and uses the LLM to create a summary, which is then used as context for the next turn. This helps the AI remember the conversation without hitting context limits.

We then create a `ConversationChain` that incorporates this summary memory. The `verbose=True` flag helps us see the prompt and the generated summary during the conversation.

In [10]:
from sqlalchemy import create_engine
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
from langchain_community.chat_message_histories import SQLChatMessageHistory

# Define a session ID for our conversation
session_id = "my_conversation_session_1"

# Set up the SQLite connection using SQLAlchemy
# The format is "sqlite:///filename.db"
db_file = "langchain_memory.db"
connection_string = f"sqlite:///{db_file}"
engine = create_engine(connection_string)

# Initialize SQLChatMessageHistory
# We now pass the connection_string directly
message_history = SQLChatMessageHistory(
    session_id=session_id, connection_string=connection_string
)

# Initialize ConversationSummaryMemory with our SQLite message history
summary_memory = ConversationSummaryMemory(
    llm=llm,
    chat_memory=message_history
)

# Create a new ConversationChain with summary memory
conversation_with_summary_memory = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=True  # Set to True to see the prompt and summary
)

# Let's have a conversation and see the summary in action
print(conversation_with_summary_memory.predict(input="Hi, I'm planning a trip to Brazil."))
print(conversation_with_summary_memory.predict(input="I want to visit Rio de Janeiro and the Amazon rainforest."))
print(conversation_with_summary_memory.predict(input="What are some must-do activities in Rio?"))

# Let's ask a follow-up question that relies on the summary
print(conversation_with_summary_memory.predict(input="Based on my plans, what's a good time of year to go?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm planning a trip to Brazil.
AI:

> Finished chain.
 That sounds exciting! Brazil is a beautiful country with a rich culture and diverse landscapes. Are you planning to visit any specific cities or regions? I can provide recommendations based on your interests and budget.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human mentions planning a trip to Brazil. The AI expresses exc